### HCP Publications Dataset
- **Description:** Human Connectome Project (HCP) publications and citation networks.
- **Data Source:**
    - points: [aggregate_titles_embeddings_umap_2d_with_info.parquet](https://www.dropbox.com/scl/fi/uj14y2hre4he2iafpativ/aggregate_titles_embeddings_umap_2d_with_info.parquet?rlkey=tjey12v6cru3iq88xitytefsr&dl=1)
    - links: [citation-links-hcp3.parquet](https://www.dropbox.com/scl/fi/3om0epfc2ukkad2ld87rs/citation-links-hcp3.parquet?rlkey=3ldck9m0hs7snswixn2qdts8i&dl=1)
- **Potential columns for visualization:**
    - **X & Y Coordinates:** `x`, `y`
    - **Point Size:** `n_cits` (citation count)
    - **Color:** `main_field` (research domain)
    - **Label:** `title`
- **Related code file:** [hcp.py](https://github.com/thorwhalen/imbed_data_prep/blob/main/imbed_data_prep/hcp.py)

## Get data

### Data parameters

In [25]:
ext = '.parquet'
src = 'https://www.dropbox.com/scl/fi/uj14y2hre4he2iafpativ/aggregate_titles_embeddings_umap_2d_with_info.parquet?rlkey=tjey12v6cru3iq88xitytefsr&dl=1'
target_filename = 'aggregate_titles_embeddings_umap_2d_with_info.parquet'

links_src = 'https://www.dropbox.com/scl/fi/3om0epfc2ukkad2ld87rs/citation-links-hcp3.parquet?rlkey=3ldck9m0hs7snswixn2qdts8i&dl=1'
links_target_filename = 'citation-links-hcp3.parquet'

### Install and import

In [2]:
import os
if not os.getenv('IN_COSMO_DEV_ENV'):
    %pip install -q cosmograph tabled cosmodata

import tabled
import cosmodata

from functools import partial 
from cosmograph import cosmo

### Load data

In [26]:
if ext:
    getter = partial(tabled.get_table, ext=ext)
else:
    getter = tabled.get_table
# acquire_data takes care of caching locally too, so next time access will be faster
# (If you want a fresh copy, you can delete the local cache file manually.)
data = cosmodata.acquire_data(src, target_filename, getter=getter)
links = cosmodata.acquire_data(links_src, links_target_filename, getter=getter)

## Peep at the data

In [4]:
mode = 'short'  #Literal['short', 'sample', 'stats'] = 'short',
exclude_cols = []
cosmodata.print_dataframe_info(data, exclude_cols, mode=mode)

DataFrame shape: (340855, 9)
First row
------------------------------------------------------------
id                                                          245658
x                                                        10.981117
y                                                          7.02786
title            Standardized low-resolution brain electromagne...
source                                                      PubMed
pub_date                                                2002-01-01
n_cits                                                        1056
micro_cluster                                                  241
main_field                          Biomedical and health sciences


In [28]:
cosmodata.print_dataframe_info(links, exclude_cols, mode=mode)

DataFrame shape: (4891305, 3)
First row
------------------------------------------------------------
citing_pub_id        245658
cited_pub_id     2013150886
date             2002-01-01


## Visualize data

### Scatter Plot of Publications

This scatter plot visualizes publications based on their `x` and `y` coordinate values. The size of each point represents the number of citations each publication has received, allowing a quick assessment of impact based on `n_cits`. Each point is colored based on the `main_field` of the publication, helping to visually distinguish the different fields of study.

In [6]:
cosmo(
    data,
    point_x_by="x",
    point_y_by="y",
    point_size_by="n_cits",
    point_color_by="main_field",
    point_id_by="id",
    point_label_by="title",
    point_color_palette=["#FF6347", "#4682B4", "#3CB371"],
    point_size_range=[5, 20],
    show_labels=True,
    render_links=False,
)

Cosmograph(background_color=None, components_display_state_mode=None, focused_point_ring_color=None, hovered_p…

### Clustered Publication Map by Micro Clusters

This visualization employs point clustering based on the `micro_cluster` column, enabling users to identify concentrations of related publications in the coordinate space defined by `x` and `y`. The points are sized by citation count, and their colors differentiate micro clusters.

In [7]:
cosmo(
    data,
    point_x_by="x",
    point_y_by="y",
    point_size_by="n_cits",
    point_color_by="micro_cluster",
    point_id_by="id",
    point_color_palette=["#FF4500", "#32CD32", "#1E90FF"],
    point_size_range=[2, 15],
    show_labels=True,
    show_top_labels=True,
    show_top_labels_limit=5,
    fit_view_on_init=True,
    disable_simulation=False,
)

Cosmograph(background_color=None, components_display_state_mode=None, disable_simulation=False, fit_view_on_in…

### Publication Network Visualization

Here we visually represent relationships between publications by treating them as nodes linked by their citation connections. This requires additional linking data to depict how these publications relate through citations, using `id` as node identifiers. Links can be created to reflect citation patterns if the data extends to include connections.

In [ ]:
print

In [30]:
cosmo(
    data,
    links=links,
    point_size_by="n_cits",
    point_color_by="main_field",
    point_id_by="id",
    link_source_by="citing_pub_id",
    link_target_by="cited_pub_id",
    link_width=1,
    link_arrows=False,
    render_links=False,
    show_labels=False,
)

Cosmograph(background_color=None, components_display_state_mode=None, focused_point_ring_color=None, hovered_p…

In [35]:
graph.capture_screenshot()